In [20]:
import folium
import folium.plugins as plugins
import pandas as pd
import json
import requests
import time
from folium.plugins import HeatMapWithTime
import datetime

In [21]:
#input how long you wish to see
mins = input('How many mins do you want to see? ')
number_of_times = (int(mins)*60)/30

In [18]:
#this is how many times the loop will pull data from the API
number_of_times

20.0

In [19]:
#the loop will pull live data every 30 secs and append it into a dataframe
cumulative = pd.DataFrame()
startTime = time.time()
for i in range(int(number_of_times)):
    url = ("https://api.data.gov.sg/v1/transport/taxi-availability")
    response = requests.get(url)
    data = response.json()
    df = pd.io.json.json_normalize(data['features'])
    coordinateslist = df['geometry.coordinates'].tolist()
    df1 = pd.DataFrame(coordinateslist)
    result = df1.transpose()
    result.columns = ['coordinates']
    result['Timestamp'] = (df['properties.timestamp'][0])
    cumulative = cumulative.append(result) 
    time.sleep(30)
endTime = time.time()
elapsedTime = endTime - startTime
print("Elapsed Time = %s" % elapsedTime,'seconds')

C:\Users\MYPC~1\AppData\Local\Temp/ipykernel_20816/401880637.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(data['features'])


Elapsed Time = 632.6120615005493 seconds


In [6]:
cumulative

,coordinates,Timestamp
0,"[103.62353, 1.29397]",2021-11-21T20:06:24+08:00
1,"[103.62469, 1.29715]",2021-11-21T20:06:24+08:00
2,"[103.62634, 1.31299]",2021-11-21T20:06:24+08:00
3,"[103.62879, 1.29885]",2021-11-21T20:06:24+08:00
4,"[103.62989, 1.28932]",2021-11-21T20:06:24+08:00
...,...,...
2587,"[103.98899, 1.36144]",2021-11-21T20:07:54+08:00
2588,"[103.9891814, 1.36070306666667]",2021-11-21T20:07:54+08:00
2589,"[103.98942, 1.35931]",2021-11-21T20:07:54+08:00
2590,"[103.989630833333, 1.36139133333333]",2021-11-21T20:07:54+08:00


In [7]:
#the data has to be formatted into a pandas dataframe from geojson format for easy manipulation
cumulative['coordinates'] = cumulative['coordinates'].astype(str)
Latitude = []
Longitude = []
coordinates = []
for i in cumulative['coordinates']:
    i = i.split(", ")
    lat = i[1][:-1]
    long = i[0][1:]
    Latitude.append(lat)
    Longitude.append(long)
cumulative['Latitude'] = Latitude
cumulative['Longitude'] = Longitude

In [8]:
cumulative

,coordinates,Timestamp,Latitude,Longitude
0,"[103.62353, 1.29397]",2021-11-21T20:06:24+08:00,1.29397,103.62353
1,"[103.62469, 1.29715]",2021-11-21T20:06:24+08:00,1.29715,103.62469
2,"[103.62634, 1.31299]",2021-11-21T20:06:24+08:00,1.31299,103.62634
3,"[103.62879, 1.29885]",2021-11-21T20:06:24+08:00,1.29885,103.62879
4,"[103.62989, 1.28932]",2021-11-21T20:06:24+08:00,1.28932,103.62989
...,...,...,...,...
2587,"[103.98899, 1.36144]",2021-11-21T20:07:54+08:00,1.36144,103.98899
2588,"[103.9891814, 1.36070306666667]",2021-11-21T20:07:54+08:00,1.36070306666667,103.9891814
2589,"[103.98942, 1.35931]",2021-11-21T20:07:54+08:00,1.35931,103.98942
2590,"[103.989630833333, 1.36139133333333]",2021-11-21T20:07:54+08:00,1.36139133333333,103.989630833333


In [9]:
cumulative['Timestamp']

0       2021-11-21T20:06:24+08:00
1       2021-11-21T20:06:24+08:00
2       2021-11-21T20:06:24+08:00
3       2021-11-21T20:06:24+08:00
4       2021-11-21T20:06:24+08:00
                  ...            
2587    2021-11-21T20:07:54+08:00
2588    2021-11-21T20:07:54+08:00
2589    2021-11-21T20:07:54+08:00
2590    2021-11-21T20:07:54+08:00
2591    2021-11-21T20:07:54+08:00
Name: Timestamp, Length: 10306, dtype: object

In [10]:
#converted Timestamp into datetime format and using it as an index
cumulative['Timestamp']= pd.to_datetime(cumulative['Timestamp'])
time_index = []
for i in cumulative['Timestamp'].unique():
    time_index.append(i)
date_strings = [dt.strftime('%d/%m/%Y, %H:%M:%S') for dt in time_index]

    

In [11]:
date_strings

['21/11/2021, 20:06:24',
 '21/11/2021, 20:06:54',
 '21/11/2021, 20:07:24',
 '21/11/2021, 20:07:54']

In [12]:
#formatting the data into a nested list 
lat_long_list = []
for i in cumulative['Timestamp'].unique():
    temp=[]
    for index, instance in cumulative[cumulative['Timestamp'] == i].iterrows():
        temp.append([instance['Latitude'],instance['Longitude']])
    lat_long_list.append(temp)
    

In [13]:
lat_long_list


[[['1.29397', '103.62353'],
  ['1.29715', '103.62469'],
  ['1.31299', '103.62634'],
  ['1.29885', '103.62879'],
  ['1.28932', '103.62989'],
  ['1.28434', '103.63078'],
  ['1.32047', '103.6309'],
  ['1.29597266666667', '103.632138'],
  ['1.32042', '103.6326'],
  ['1.29757', '103.63298'],
  ['1.30937', '103.63597'],
  ['1.33408', '103.6432'],
  ['1.32825', '103.64844'],
  ['1.3144', '103.65565'],
  ['1.31622', '103.66212'],
  ['1.31076', '103.66221'],
  ['1.31231', '103.66247'],
  ['1.32208', '103.66403'],
  ['1.33', '103.6777'],
  ['1.33034', '103.67833'],
  ['1.31565', '103.67885'],
  ['1.31568', '103.67896'],
  ['1.32803498333333', '103.6794157'],
  ['1.33008', '103.68234'],
  ['1.31962', '103.68509'],
  ['1.34098883333333', '103.688757333333'],
  ['1.34229', '103.69031'],
  ['1.34123281666667', '103.690492516667'],
  ['1.34171', '103.6906'],
  ['1.34128', '103.6913'],
  ['1.34176', '103.69184'],
  ['1.34', '103.69249'],
  ['1.34', '103.69276'],
  ['1.34', '103.69346'],
  ['1.34283', 

In [14]:
#Choosing the map type 
m = folium.Map(location=[1.352083,103.819839],zoom_start = 11, tiles="https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png",attr="Stadia.AlidadeSmoothDark") 

#Plot it on the map
HeatMapWithTime(lat_long_list,radius=5,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)

# Display the map
m


In [15]:
#saving it as a HTML file 
m.save('a.html')